# <h1 style = 'font-family:Monocraft'>Swinburne FAQ Chatbot Scope Detection Component</h1>

## <h2 style='font-family:Monocraft'>Scope Training</h2>
    randomly shuffles 4 question label datasets to include 90% in scope and 10% out of scope data, then loads pretrained model and continuously trains them on randomised datasets.

In [10]:
import random
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DistilBertForSequenceClassification, \
    Trainer, TrainingArguments, DataCollatorWithPadding

#set iterations. each iteration will have different out-of-scope training data.
iterations = 2

for iteration in range (iterations):

    #load datasets
    faq_data = pd.read_csv('scope_training_datasets/faq_data_batch_1.csv')
    faq_data2 = pd.read_csv('scope_training_datasets/faq_data_batch_2.csv')
    faq_data3 = pd.read_csv('scope_training_datasets/altered_phung_dataset.csv')
    faq_data4 = pd.read_csv('scope_training_datasets/jokes_set.csv')
    faq_data5 = pd.read_csv('scope_training_datasets/testdata.csv')

    #splits into in scope and out of scope, takes random out of-scope data until total with in scope results to 90-10% split
    in_scope_data = faq_data[:685]
    out_of_scope_data = faq_data[685:]

    num_out_of_scope_for_training = int(80)
    out_of_scope_training_data = out_of_scope_data.sample(n=num_out_of_scope_for_training, random_state=42)

    in_scope_data2 = faq_data2[:685]
    out_of_scope_data2 = faq_data2[685:]

    num_out_of_scope_for_training2 = int(80)
    out_of_scope_training_data2 = out_of_scope_data2.sample(n=num_out_of_scope_for_training2, random_state=42)

    in_scope_data3 = faq_data3[:1027]
    out_of_scope_data3 = faq_data3[1027:]

    num_out_of_scope_for_training3 = int(100)
    out_of_scope_training_data3 = out_of_scope_data3.sample(n=num_out_of_scope_for_training3, random_state=42)

    in_scope_data4 = faq_data4[:685]
    out_of_scope_data4 = faq_data4[685:]

    num_out_of_scope_for_training4 = int(100)
    out_of_scope_training_data4 = out_of_scope_data3.sample(n=num_out_of_scope_for_training4, random_state=42)

    in_scope_data5 = faq_data5[:41]
    out_of_scope_data5 = faq_data5[41:]

    #concatenate the in-scope and out-of-scope training data
    training_data = pd.concat([in_scope_data, in_scope_data2, out_of_scope_training_data, out_of_scope_training_data2, in_scope_data3, out_of_scope_training_data3, in_scope_data4, out_of_scope_training_data4, in_scope_data5, out_of_scope_data5])

    #convert the training data to huggingface dataset format
    training_dataset = Dataset.from_pandas(training_data)

    #renames columns to suit trainer
    training_dataset = training_dataset.rename_column("Question", "text")
    training_dataset = training_dataset.rename_column("Label", "label")

    #map labels to 0 for in-scope and 1 for out-of-scope
    training_dataset = training_dataset.map(lambda example: {"label": 1 if example["label"] == 1 else 0}, 
                              remove_columns=["label"])

    #tokenise training set
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized_training_dataset = training_dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True)

    #split the tokenized training dataset into training and validation sets (10% eval, 90% train)
    split_tokenized_training_dataset = tokenized_training_dataset.train_test_split(test_size=0.1)
    train_dataset = split_tokenized_training_dataset["train"]
    eval_dataset = split_tokenized_training_dataset["test"]

    #load model and collater
    distilbert_model = DistilBertForSequenceClassification.from_pretrained('./faq_scope_detection', num_labels=2)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    #training params
    training_args = TrainingArguments(
        output_dir='./faq_scope_detection',
        num_train_epochs=5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./faq_scope_detection_logs',
        logging_steps=10,
        save_steps=50,
        evaluation_strategy='epoch',
        learning_rate=2e-5,
    )

    #initialise and run trainer
    trainer = Trainer(
        model=distilbert_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    trainer.train()

    #overwrite model
    trainer.save_model('./faq_scope_detection')


  4%|▎         | 59/1620 [55:02<24:16:03, 55.97s/it]
loading configuration file config.json from cache at /Users/justin/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/justin/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache a

{'loss': 0.5184, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.03}


  1%|          | 20/1995 [00:04<07:56,  4.15it/s]

{'loss': 0.3007, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.05}


  2%|▏         | 31/1995 [00:07<07:05,  4.61it/s]

...

100%|█████████▉| 1990/1995 [08:58<00:01,  3.93it/s]

{'loss': 0.0, 'learning_rate': 6.688963210702342e-08, 'epoch': 4.99}


100%|██████████| 1995/1995 [08:59<00:00,  3.95it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Answer, text, __index_level_0__. If Answer, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 355
  Batch size = 8

100%|██████████| 1995/1995 [09:01<00:00,  3.95it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1995/1995 [09:01<00:00,  3.68it/s]
Saving model checkpoint to ./faq_scope_detection
Configuration saved in ./faq_scope_detection/config.json


{'eval_loss': 1.3432031042981407e-09, 'eval_runtime': 2.3013, 'eval_samples_per_second': 154.263, 'eval_steps_per_second': 19.555, 'epoch': 5.0}
{'train_runtime': 541.7982, 'train_samples_per_second': 29.457, 'train_steps_per_second': 3.682, 'train_loss': 1.053677208203838e-06, 'epoch': 5.0}


Model weights saved in ./faq_scope_detection/pytorch_model.bin
tokenizer config file saved in ./faq_scope_detection/tokenizer_config.json
Special tokens file saved in ./faq_scope_detection/special_tokens_map.json


In [50]:
import random
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DistilBertForSequenceClassification, \
    Trainer, TrainingArguments, DataCollatorWithPadding

#set iterations. each iteration will have different out-of-scope training data.
iterations = 1

for iteration in range (iterations):

    #load datasets
    faq_data = pd.read_csv('scope_training_datasets/swinburneusage.csv')

    #splits into in scope and out of scope, takes random out of-scope data until total with in scope results to 90-10% split
    in_scope_data = faq_data[:13]
    out_of_scope_data = faq_data[13:]


    #concatenate the in-scope and out-of-scope training data
    training_data = pd.concat([in_scope_data, out_of_scope_data])

    #convert the training data to huggingface dataset format
    training_dataset = Dataset.from_pandas(training_data)

    #renames columns to suit trainer
    training_dataset = training_dataset.rename_column("Question", "text")
    training_dataset = training_dataset.rename_column("Label", "label")

    #map labels to 0 for in-scope and 1 for out-of-scope
    training_dataset = training_dataset.map(lambda example: {"label": 1 if example["label"] == 1 else 0}, 
                              remove_columns=["label"])

    #tokenise training set
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized_training_dataset = training_dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True)

    #split the tokenized training dataset into training and validation sets (10% eval, 90% train)
    split_tokenized_training_dataset = tokenized_training_dataset.train_test_split(test_size=0.1)
    train_dataset = split_tokenized_training_dataset["train"]
    eval_dataset = split_tokenized_training_dataset["test"]

    #load model and collater
    distilbert_model = DistilBertForSequenceClassification.from_pretrained('./faq_scope_detection', num_labels=2)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    #training params
    training_args = TrainingArguments(
        output_dir='./faq_scope_detection',
        num_train_epochs=5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./faq_scope_detection_logs',
        logging_steps=10,
        save_steps=50,
        evaluation_strategy='epoch',
        learning_rate=2e-5,
    )

    #initialise and run trainer
    trainer = Trainer(
        model=distilbert_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    trainer.train()

    #overwrite model
    trainer.save_model('./faq_scope_detection')


loading configuration file config.json from cache at /Users/justin/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/justin/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /Users/justin/.cache/huggingface/hub/models--distil

{'eval_loss': 12.068842887878418, 'eval_runtime': 0.0326, 'eval_samples_per_second': 92.039, 'eval_steps_per_second': 30.68, 'epoch': 1.0}


 40%|████      | 6/15 [00:01<00:02,  4.24it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3
  Batch size = 8

 40%|████      | 6/15 [00:01<00:02,  4.24it/s]

{'eval_loss': 11.965846061706543, 'eval_runtime': 0.0298, 'eval_samples_per_second': 100.556, 'eval_steps_per_second': 33.519, 'epoch': 2.0}


 60%|██████    | 9/15 [00:02<00:01,  4.36it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3
  Batch size = 8

 60%|██████    | 9/15 [00:02<00:01,  4.36it/s]

...

 80%|████████  | 12/15 [00:02<00:00,  4.40it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3
  Batch size = 8

 80%|████████  | 12/15 [00:02<00:00,  4.40it/s]

{'eval_loss': 11.539115905761719, 'eval_runtime': 0.0321, 'eval_samples_per_second': 93.493, 'eval_steps_per_second': 31.164, 'epoch': 4.0}


100%|██████████| 15/15 [00:03<00:00,  4.42it/s]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3
  Batch size = 8

100%|██████████| 15/15 [00:03<00:00,  4.42it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 15/15 [00:03<00:00,  4.14it/s]
Saving model checkpoint to ./faq_scope_detection
Configuration saved in ./faq_scope_detection/config.json


{'eval_loss': 11.224631309509277, 'eval_runtime': 0.0305, 'eval_samples_per_second': 98.473, 'eval_steps_per_second': 32.824, 'epoch': 5.0}
{'train_runtime': 3.6155, 'train_samples_per_second': 29.041, 'train_steps_per_second': 4.149, 'train_loss': 4.308137003580729, 'epoch': 5.0}


Model weights saved in ./faq_scope_detection/pytorch_model.bin
tokenizer config file saved in ./faq_scope_detection/tokenizer_config.json
Special tokens file saved in ./faq_scope_detection/special_tokens_map.json


## <h2 style='font-family:Monocraft'>Model Evaluation Script<h2>
    calcs f1 and accuracy using a test dataset and prints. also prints incorrectly labelled questions.

In [1]:
import pandas as pd
from transformers import AutoTokenizer, DistilBertForSequenceClassification
import torch

#load model
tokenizer = AutoTokenizer.from_pretrained('./scopeclassifier')
model = DistilBertForSequenceClassification.from_pretrained('./scopeclassifier')

#load test set
test_data = pd.read_csv('scope_training_datasets/testdata.csv')

#name columns
test_data = test_data.rename(columns={'Question': 'text'})
test_data = test_data[:1000]
#tokenise dataset
tokenized_test_dataset = tokenizer(test_data['text'].tolist(), truncation=True, padding=True, return_tensors='pt')

#functions for f1 and acc
def calculate_accuracy(preds, labels):
    correct = (preds == labels).sum().item()
    total = labels.size(0)
    return correct / total

def calculate_f1_score(preds, labels):
    tp = ((preds == 1) & (labels == 1)).sum().item()
    tn = ((preds == 0) & (labels == 0)).sum().item()
    fp = ((preds == 1) & (labels == 0)).sum().item()
    fn = ((preds == 0) & (labels == 1)).sum().item()

    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)

    return f1

#generate predictions
model.eval()
with torch.no_grad():
    outputs = model(**tokenized_test_dataset)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

#get labels
labels = torch.tensor(test_data['Label'].tolist())

#calc f1 and acc and print scores
accuracy = calculate_accuracy(predictions, labels)
f1_score = calculate_f1_score(predictions, labels)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")

#print failures
incorrect_idx = (predictions != labels).nonzero().squeeze()
incorrect_predictions = predictions[incorrect_idx].tolist()
true_labels = labels[incorrect_idx].tolist()
incorrect_questions = test_data.iloc[incorrect_idx]['text'].tolist()
print(incorrect_idx.size())

print("\nInaccurately labeled questions:")
for q, p, l in zip(incorrect_questions, incorrect_predictions, true_labels):
    print(f"Question: {q}")
    print(f"Predicted label: {p}")
    print(f"True label: {l}")
    print()

/Users/justin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accuracy: 1.0
F1 Score: 0.999999994846154
torch.Size([0])

Inaccurately labeled questions:
